In [208]:
%reset -f

import sys
import os
import csv
import pandas as pd
import numpy as np
import pathlib
from tqdm import tqdm

from events_compare import events_to_df
from events_compare import events_compare

OTA_events, GTCounts, Eventcounts = events_compare("Z:/04_Daten/GroundThruth/Saarbruecken")
del OTA_events['Tail'], OTA_events['Tail2'], OTA_events['occurrence'], OTA_events['fps'], OTA_events['video_name'], OTA_events['event_coordinate'], OTA_events['direction_vector']

# print(Eventcounts[Eventcounts['road_user_type'] == 'pedestrian'])

# Tracks identifizieren, die eine section mehrmals schneiden
OTA_events['event_type2'] = np.where(OTA_events['event_type']== 'section-enter', 'section_' + OTA_events['section_id'], OTA_events['event_type'])
eventtypecount = pd.DataFrame(OTA_events.groupby(['road_user_id', 'event_type2'])['event_type2'].count()).rename(columns = {'event_type2': 'Event_Type_Count'}).reset_index()
eventtypecount = eventtypecount.pivot(index='road_user_id', columns='event_type2', values='Event_Type_Count').rename(columns = {'enter-scene': 'count_enter-scene', 'leave-scene': 'count_leave-scene', 'section_7': 'count_section_7', 'section_8': 'count_section_8', }).reset_index().fillna(0)
OTA_events = pd.merge(OTA_events, eventtypecount, how='outer', left_on='road_user_id', right_on='road_user_id')

# Frame number der Events dazupacken (wide)
frame_number = OTA_events[['road_user_id', 'frame_number', 'event_type2']]
frame_number = pd.DataFrame(frame_number.groupby(['road_user_id', 'event_type2'])['frame_number'].min()).reset_index()
frame_number = frame_number.sort_values(['road_user_id', 'frame_number'], ascending=True).drop_duplicates()
frame_number = frame_number.pivot(index='road_user_id', columns='event_type2', values='frame_number').rename(columns = {'enter-scene': 'frame_enter-scene', 'leave-scene': 'frame_leave-scene', 'section_7': 'frame_section_7', 'section_8': 'frame_section_8', }).reset_index().reset_index().fillna(0)
OTA_events = pd.merge(OTA_events, frame_number, how='outer', left_on='road_user_id', right_on='road_user_id')
OTA_events['section_id'] = OTA_events['section_id'].fillna(0)

# Jede road_user_id nur einmal um mehrere gleich Events zu filtern
OTA_events = OTA_events.drop_duplicates('road_user_id')

# Counts
OTA_events['Trackdauer'] = OTA_events['frame_leave-scene'] - OTA_events['frame_enter-scene']
OTA_events['Flowdauer'] = OTA_events['frame_section_8'] - OTA_events['frame_section_7']

## Track-Klassifikation
section7 = OTA_events['frame_section_7']
section8 = OTA_events['frame_section_8']

conditions = [
    (section7 != 0) & (section8 != 0) & (section7 <= section8),
    (section7 != 0) & (section8 != 0) & (section7 > section8),
    (section7 != 0) & (section8 == 0),
    (section7 == 0) & (section8 != 0),
    (section7 == 0) & (section8 == 0)
]

choices = [
    'Flow 7->8',
    'Flow 8->7',
    'Section 7',
    'Section 8',
    'Single Event'
    
]

OTA_events['Klassifikation'] = np.select(conditions, choices, 'Keine Ahnung')

OTACounts = OTA_events.groupby(['OTCamera', 'Date', 'Time', 'road_user_type', 'Klassifikation'])['road_user_id'].count().reset_index().rename(columns={'road_user_id': 'Count_OTA'})

Counts = pd.merge(OTACounts, GTCounts, how='outer', left_on=['Date', 'Time', 'section_id', 'road_user_type', 'OTCamera'], right_on = ['Date', 'Time', 'section_id', 'road_user_type', 'OTCamera'])
Counts['Differenz'] = Counts['OTGroundTruth'] - Counts['OTAnalytics_0']
Counts['Diff [%]'] = np.round((1 - np.round(Counts['OTAnalytics_0'] / Counts['OTGroundTruth'], 3)) * 100, 1)


KeyError: 'section_id'

- road_user_id = 1caa258b-bf76-4960-8fd0-d105b2d816dc#2#1027
- frames 11010 - 11058
- 5 mal section-enter
- Muss am Tracking liegen, alle Bounding Boxes der Personengruppe werden zusammengeschmissen